# Problem 2

Use this notebook to write your code for problem 3.

In [78]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, LSTM, LeakyReLU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

# Load the training and test data
data_train = load_data('data/training_data.txt', 1)
X_train = data_train[:, 1:]
# Normalize the data
#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)

# One-hot encode the labels
y_train_pre = data_train[:, 0]
y_train = keras.utils.np_utils.to_categorical(y_train_pre,num_classes=2)
#y_train = y_train_pre

data_test = load_data('data/test_data.txt', 1)
X_test = data_test[:,:]
#X_test = scaler.transform(X_test)

## 2C - Depth vs Width for MNIST

As in problem 2, we have conveniently provided for your use code that loads, preprocesses, and deals with the uglies of the MNIST data.

In [79]:
#X_train[np.where(X_train>1)] = X_train[np.where(X_train>1)] +1;
#X_test[np.where(X_test>1)] = X_test[np.where(X_test>1)] +1;

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
pca = PCA(n_components = 'mle', svd_solver = 'full')
pca.fit(X_train)
pca.transform(X_train)

In [75]:

def create_model():
    
    model = Sequential()
    model.add(Dense(50,input_shape=(1000,)))
    #model.add(LeakyReLU(alpha=0.))
    #model.add(Dense(1000,input_shape=(1000,)))
    #model.add(Activation('relu'))
    model.add(Dense(100))
    model.add(Dropout(0.05))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model
    #model.summary()


In [76]:
## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta

from sklearn.model_selection import train_test_split

kfold = 1
if kfold == 1:
    n_splits = 5
    skf = StratifiedKFold(shuffle=True, n_splits=n_splits)

    pred = np.zeros(10000)
    i = 0
    for train_index, test_index in skf.split(X_train, y_train_pre):
        print("Running Fold", i+1, "/", n_splits)
        model = None 
        model = create_model()

        model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

        fit = model.fit(X_train[train_index], y_train[train_index], batch_size=128, epochs=15, verbose=1)    
        score = model.evaluate(X_train[test_index], y_train[test_index], verbose=0)

        pred += model.predict(X_test)[:,1]

        print('Test score:', score[0])
        print('Test accuracy:', score[1])
        i += 1

    y_labs = np.maximum(0.,np.sign(pred-2.5))
else:
    X_train_t, X_train_v, y_train_t, y_train_v = train_test_split(X_train,y_train,test_size=0.2)

    model = None 
    model = create_model()

    model.compile(loss='hinge',optimizer='adam', metrics=['accuracy'])

    fit = model.fit(X_train_t, y_train_t, batch_size=64, epochs=10, verbose=1)    
    score = model.evaluate(X_train_v, y_train_v, verbose=0)

    pred = model.predict(X_test)[:,1]

    print('Test score:', score[0])
    print('Test accuracy:', score[1])


    y_labs = np.maximum(0.,np.sign(pred-0.5))
    

Running Fold 1 / 5
Epoch 1/15
15999/15999 [==============================] - 2s 120us/step - loss: 0.4860 - acc: 0.7812
Epoch 2/15
15999/15999 [==============================] - 1s 38us/step - loss: 0.3632 - acc: 0.8471
Epoch 3/15
15999/15999 [==============================] - 1s 47us/step - loss: 0.3483 - acc: 0.8519
Epoch 4/15
15999/15999 [==============================] - 1s 44us/step - loss: 0.3434 - acc: 0.8549
Epoch 5/15
15999/15999 [==============================] - 1s 43us/step - loss: 0.3343 - acc: 0.8569
Epoch 6/15
15999/15999 [==============================] - 1s 41us/step - loss: 0.3331 - acc: 0.8596
Epoch 7/15
15999/15999 [==============================] - 1s 43us/step - loss: 0.3295 - acc: 0.8619
Epoch 8/15
15999/15999 [==============================] - 1s 41us/step - loss: 0.3262 - acc: 0.8626
Epoch 9/15
15999/15999 [==============================] - 1s 41us/step - loss: 0.3264 - acc: 0.8632
Epoch 10/15
15999/15999 [==============================] - 1s 40us/step - loss: 

KeyboardInterrupt: 

In [ ]:
file = open("labels4.txt","w") 
file.write("Id,Prediction\n")
for i,x in enumerate(y_labs):
    file.write("%i,%i\n" % (i+1,x))

In [ ]:
model.summary()